In [ ]:
import numpy as np
import pickle
from astropy.io import fits
import matplotlib.pyplot as plt
from galpy.util import bovy_conversion, bovy_coords, save_pickles, bovy_plot
from galpy.potential import MWPotential2014, turn_physical_off, vcirc
import astropy.units as u
%pylab inline
from galpy.orbit import Orbit
import GMC_util
import pal5_util


ro=8.
#paper on MC used R0=8.5 kpc, using ro=8. as of now.
vo=220.

In [ ]:
timpact,apar,x_stream,y_stream,z_stream,_,_,_=GMC_util.aparxv_stream_from_pkl(sampling=2048,nchunks=1)

In [ ]:
print (apar)

In [ ]:
impactMC_ind,M_mc,rs_mc,v_mc,impactb,impact_angle,tmin=GMC_util.compute_impact_parameters(timpact,apar,x_stream,y_stream,z_stream,nchunks=1,imp_fac=5.,Mmin=10**6.)

In [ ]:
print (tmin)

In [ ]:
#load the lower timpact pkl file
with open('pkl_files/pal5pepper_128sampling_MW2014.pkl','rb') as savefile:
            sdf_pepper= pickle.load(savefile,encoding='latin1')
        
sdf_smooth= pal5_util.setup_pal5model()

sdf_pepper.set_impacts(impactb=impactb,subhalovel=v_mc,impact_angle=impact_angle,timpact=tmin,rs=rs_mc,GM=M_mc)




In [ ]:
apar=np.arange(0.,1.75,0.01)

densOmega= numpy.array([sdf_pepper._densityAndOmega_par_approx(a) for a in apar]).T

dens=densOmega[0]
Omega=densOmega[1]

dens_unp= [sdf_smooth._density_par(a) for a in apar]

In [ ]:
plt.figure()
plt.plot(apar,dens/dens_unp)

plt.xlabel(r'$\theta_{\parallel}$')
plt.ylabel(r'density/smooth density')
#plt.savefig('plots/MC_106-107Msun_den_MCs_{}timpacts.pdf'.format(len(timpact)))
plt.show()

In [ ]:
timp=sdf_pepper._timpact

impactMC_ind,M_mc,rs_mc,v_mc,impactb,impact_angle,tmin_impact=compute_impact_parameters_one_pkl(timp,apar,x,y,z)

In [ ]:
sdf_pepper.set_impacts(impactb=impactb,subhalovel=v_mc,impact_angle=impact_angle,timpact=tmin_impact,rs=rs_mc,GM=M_mc)

In [ ]:
#load the lower timpact pkl file
with open('pkl_files/pal5pepper_128sampling_MW2014.pkl','rb') as savefile:
            sdf_pepper_128= pickle.load(savefile,encoding='latin1')

sdf_smooth= pal5_util.setup_pal5model()        
#set impacts
sdf_pepper_128.set_impacts(impactb=impactb,subhalovel=v_mc,impact_angle=impact_angle,timpact=tmin_low,rs=rs_mc,GM=M_mc)

In [ ]:
apar=np.arange(0.,1.75,0.01)


densOmega= numpy.array([sdf_pepper_128._densityAndOmega_par_approx(a) for a in apar]).T

dens=densOmega[0]
Omega=densOmega[1]

dens_unp= [sdf_smooth._density_par(a) for a in apar]

In [ ]:
import pal5_util
from scipy import ndimage, signal, interpolate

#with open('Pal5_64impact_MC.pkl','rb') as savefile:
#            sdf_smooth=pickle.load(savefile,encoding='latin1')
#            sdf_pepper= pickle.load(savefile,encoding='latin1')


def convert_dens_to_obs(apars,dens,dens_smooth,mO,sdf_pepper,poly_deg=3):
    """
    NAME:
        convert_dens_to_obs
    PURPOSE:
        Convert track to observed coordinates
    INPUT:
        apars - parallel angles
        dens - density(apars)
        dens_smooth - smooth-stream density(apars)
        mO= (None) mean parallel frequency (1D) 
            [needs to be set to get density on same grid as track]
        poly_deg= (3) degree of the polynomial to fit for the 'smooth' stream
    OUTPUT:
        (xi,dens/smooth)
    """
    mT= sdf_pepper.meanTrack(apars,_mO=mO,coord='lb')
    mradec= bovy_coords.lb_to_radec(mT[0],mT[1],degree=True)
    mxieta= pal5_util.radec_to_pal5xieta(mradec[:,0],mradec[:,1],degree=True)
    outll= numpy.arange(0.25,14.35,0.1)
    # Interpolate density
    ipll= interpolate.InterpolatedUnivariateSpline(mxieta[:,0],apars)
    ipdens= interpolate.InterpolatedUnivariateSpline(apars,dens/dens_smooth)
    return (outll,ipdens(ipll(outll)))


def set_ranges_and_labels_dens():
    bovy_plot.bovy_text(r'$\sqrt{\delta\delta}$',top_left=True,size=18.)
    xlabel(r'$1/k_{\xi}\,(\mathrm{deg})$',size=18)
    ylim(0.0001,10.)
    xlim(1,150.)
    for axis in [gca().xaxis,gca().yaxis]:
        axis.set_major_formatter(FuncFormatter(
                lambda y,pos: (r'${{:.{:1d}f}}$'.format(int(numpy.maximum(-numpy.log10(y),0)))).format(y)))
    return None

In [ ]:
dat256=np.loadtxt('dens_MC_impact_Pal5_256timpacts.dat')

ll,tdata= convert_dens_to_obs(apar,dens,dens_unp,Omega,sdf_pepper=sdf_pepper_128,poly_deg=3)

plt.figure()
plt.plot(ll,tdata,label='256 on 128')
plt.plot(dat256[:,0],dat256[:,1],label='full 256')

plt.xlabel(r'$\theta_{\parallel}$')
plt.ylabel(r'density/smooth density')
plt.legend(loc='lower right')
plt.savefig('plots/MC_106-107Msun_den_MCs_256timpactsfullvs_on128.pdf'.format(len(timpact)))
plt.show()

# Using impactb,velocity and impact angle of the low resolution, test to see if this produces the same results as the full sdf_pepper


In [ ]:
#from the constructed matrices of min_sep and min_apar, compute the min separation for each MC for all timpact
#and the corresponding apar_min, also compute velocity of the MC at that timpact


#
#use timpact_at_min_low to compute (x,v) of MC'

#impactMC_ind,M_mc,rs_mc,v_mc,impactb,impact_angle,tmin_low


#load the GMCs
M,rs,coord=GMC_util.add_MCs()

#integrate their orbits 5 Gyr back,
t_age= np.linspace(0.,5.,1001)/bovy_conversion.time_in_Gyr(vo,ro)

orbits=[]

N=len(M)

for ii in range(N):

    orbits.append(Orbit(coord[ii]).flip()) # flip flips the velocities for backwards integration
    orbits[ii].integrate(t_age,MWPotential2014)

impactb128=[]
impact_angle128=[]
vx_mc128=[]
vy_mc128=[]
vz_mc128=[]


sdf_smooth= pal5_util.setup_pal5model()
with open('pkl_files/pal5pepper_128sampling_MW2014.pkl','rb') as savefile:
        sdf_pepper_128= pickle.load(savefile,encoding='latin1')

timpact128=sdf_pepper_128._timpact

#print (timpact128)

ap_128,x_128,y_128,z_128,_,_,_= GMC_util.aparxv_stream(sdf_smooth,sdf_pepper_128)



#compute min_sep of each MC
for kk in range(len(tmin_low)):
    
    #find the index in timpact128 at which min sep was reached
    t_ind=list(timpact128).index(tmin_low[kk])
    t128=timpact128[t_ind]
    
    print (t128,tmin_low[kk])
    print (impactMC_ind[kk])
    x_mc=orbits[impactMC_ind[kk]].x(t128)
    y_mc=orbits[impactMC_ind[kk]].y(t128)
    z_mc=orbits[impactMC_ind[kk]].z(t128)
        
    min_sep_128,apar_min_128=GMC_util.compute_min_separation(x_mc,y_mc,z_mc,ap_128[t_ind],x_128[t_ind],y_128[t_ind],z_128[t_ind])
        
    impactb128.append(min_sep_128)
    impact_angle128.append(apar_min_128)
    #flip velocities
    vx_mc128.append(-orbits[impactMC_ind[kk]].vx(t128))
    vy_mc128.append(-orbits[impactMC_ind[kk]].vy(t128))
    vz_mc128.append(-orbits[impactMC_ind[kk]].vz(t128))

v_mc128=np.c_[vx_mc128,vy_mc128,vz_mc128]
        




In [ ]:
print (v_mc-v_mc128)

print (np.array(impact_angle)-np.array(impact_angle128))



In [ ]:
sdf_pepper_128.set_impacts(impactb=impactb128,subhalovel=v_mc128,impact_angle=impact_angle128,timpact=tmin_low,rs=rs_mc,GM=M_mc)

#compute dens in smaller intervals separately and then combine

apar=np.arange(0.,1.75,0.01)


densOmega= numpy.array([sdf_pepper_128._densityAndOmega_par_approx(a) for a in apar]).T

dens=densOmega[0]
Omega=densOmega[1]

dens_unp= [sdf_smooth._density_par(a) for a in apar]

In [ ]:
plt.figure()
plt.plot(apar,dens/dens_unp)

plt.xlabel(r'$\theta_{\parallel}$')
plt.ylabel(r'density/smooth density')
#plt.savefig('plots/MC_106-107Msun_den_MCs_{}timpacts.pdf'.format(len(timpact)))
plt.show()

In [ ]:
#write xi dens/dens_smooth out

ll,tdata= convert_dens_to_obs(apar,dens,dens_unp,Omega,poly_deg=3)

fo=open('dens_MC_impact_Pal5_256timpacts_from128sdf_pepper.dat','w')
fo.write('#xi   dens/dens_smooth' + '\n')

for ii in range(len(ll)):
    fo.write(str(ll[ii]) + '   ' + str(tdata[ii]) + '\n')
    
fo.close()

In [ ]:
def compute_Pk(filename=''):
    dat=np.loadtxt(filename)
    ll=dat[:,0]
    tdata=dat[:,1]
    px,py= signal.csd(tdata,tdata,fs=1./(ll[1]-ll[0]),scaling='spectrum',nperseg=len(ll))
    py= py.real
    px= 1./px
    py= numpy.sqrt(py*(ll[-1]-ll[0]))
    #return 180./px/numpy.pi, py, py_err #if apars
    
    return (px,py)


f2='dens_MC_impact_Pal5_256timpacts.dat'
f3='dens_MC_impact_Pal5_256timpacts_from128sdf_pepper.dat'



px2,py2=compute_Pk(filename=f2)
px3,py3=compute_Pk(filename=f3)
   


bovy_plot.bovy_print(axes_labelsize=18.,xtick_labelsize=14.,ytick_labelsize=14.)
figsize(7,8)


loglog(px2,py2,color='r',zorder=1,label=r'$10^{6} - 10^{7} \ \rm{M}_{\odot}$ 256 timpact')
loglog(px3,py3,color='b',zorder=1,label=r'$10^{6} - 10^{7} \ \rm{M}_{\odot}$ 256 timpact on 128sdfpepper')


legend(loc='lower right',fontsize=16)
set_ranges_and_labels_dens()
savefig('power_MC_Pal5_timpact256_compare_256from128sdf_pepper.pdf')